## Imports

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

n_rows =5
dataset = 
# Get batch
inputs, labels = None, None
for elem in dataset:
    inputs, labels = elem
    break
fig = plt.figure(figsize=(15,6.5))
# Variables for controllong the color map for the fire masks
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
# Number of data variables
n_features = 12
for i in range(n_rows):
    for j in range(n_features + 1):
        plt.subplot(n_rows, n_features + 1, i * (n_features + 1) + j + 1)
        if i == 0:
            plt.title(TITLES[j], fontsize=13)
        if j < n_features - 1:
            plt.imshow(inputs[i, :, :, j], cmap='viridis')
        if j == n_features - 1:
            plt.imshow(inputs[i, :, :, -1], cmap=CMAP, norm=NORM)
        if j == n_features:
            plt.imshow(labels[i, :, :, 0], cmap=CMAP, norm=NORM)
        plt.axis('off')
plt.tight_layout()


ModuleNotFoundError: No module named 'data_utils'